In [1]:
# coding: utf-8
from typing import Dict, List

import pandas as pd


class Datos:
    datos: int
    nominalAtributos: List[bool]

    # Constructor: procesar el fichero para asignar correctamente las variables nominalAtributos, datos y diccionarios
    def __init__(self, nombreFichero: str):
        self.datosCrudos = pd.read_csv(nombreFichero)
        self.datos = self.datosCrudos.copy()

        self.nominalAtributos = []
        self.diccionarios = {}

        for columna in self.datos.columns:
            if self._es_nominal(columna):
                self.nominalAtributos.append(True)
                self.diccionarios[columna] = self._generar_mapeo(columna)
                self.datos[columna] = self._reemplazar_valores(columna)
            elif self._es_numerico(columna):
                self.nominalAtributos.append(False)
                self.diccionarios[columna] = {}
            else:
                raise ValueError(
                    f"La columna '{columna}' contiene valores que no son nominales ni enteros/decimales."
                )

    def _es_nominal(self, columna: str) -> bool:
        # es verdadero si es la columna objetivo o si sus valores son nominales
        return columna.lower() == "class" or self.datos[columna].dtype.name == "object"

    def _es_numerico(self, columna: str) -> bool:
        # es verdadero si los valores son un n�meros enteros o reales
        return self.datos[columna].dtype.name in ["int64", "float64"]

    def _generar_mapeo(self, columna_nominal: str):
        # se extraen los valores �nicos de la columna y se sortean lexicograficamente
        valores = [str(valor) for valor in self.datos[columna_nominal].unique()]
        valores = sorted(valores)

        return {valor: indice for indice, valor in enumerate(valores)}

    def _reemplazar_valores(self, columna_nominal: str) -> pd.Series:
        mapeo = self.diccionarios[columna_nominal]
        return self.datos[columna_nominal].map(lambda valor: mapeo[str(valor)])

    # Devuelve el subconjunto de los datos cuyos �ndices se pasan como argumento
    def extraeDatos(self, idx: List[int]):
        return self.datos.iloc[idx]


In [47]:
import math
import random
import numpy as np
datos = Datos("wdbc.csv").datos


# normalizar los datos
for col in datos.columns:
    datos[col] = datos[col] / datos[col].abs().max()

#datos.head(5)

# crea umbral
umbral = [random.uniform(-0.5,0.5) for col in datos.columns]
umbral = umbral[:-1]

# constante de aprendizaje
k_apre = 1

# entra w.x
def sigma(w_v, x_v):
    x = np.array(x_v)[0]
    w = np.array(w_v)[0]
    print(w, x)
    1 / 1 + math.exp(-w*x)


def actualiza_w(w, x):
    t = x.iloc[:,-1:]
    x = x.iloc[:,:-1]
    print(w)
    return w - k_apre * x * (sigma(w, x) - t)

print(actualiza_w(umbral, datos.head(1)))

#print(np.array(datos.head(1))[0])


[-0.0031780974693649267, -0.2934956627407229, 0.3250082571265662, -0.31809149751059085, 0.4587299098595584, -0.11560375790639199, 0.03239864739490517, -0.4267375427591611, 0.47212533813179136, -0.38325786377470783, 0.36136033300512094, -0.04118383000115988, -0.20338831413543723, -0.20384928265178004, -0.4685186533910447, 0.10839652022143409, -0.2941077602576263, -0.49493955953806046, -0.37531003659515516, -0.011382926660737924, -0.4686530365693251, -0.08156677083771025, 0.2767084111686321, 0.4005028751793944, 0.04098068906032182, 0.13784736714719015, 0.40815878423887786, 0.3850419186123675, 0.16554697326315304, 0.2861607224671875]
-0.0031780974693649267 [0.63998577 0.26425662 0.65145889 0.4002399  0.7246022  0.80370585
 0.70313964 0.73111332 0.79572368 0.80777915 0.3811347  0.18532242
 0.39076433 0.28292143 0.20555734 0.36218612 0.13568182 0.30062512
 0.38036732 0.20754021 0.70421754 0.34981833 0.73487261 0.47461213
 0.72866128 0.62911153 0.56861022 0.91202749 0.69313046 0.57301205]


TypeError: only length-1 arrays can be converted to Python scalars